In [1]:
debug = False

from picamera import PiCamera
from picamera.array import PiRGBArray, PiYUVArray
from scipy.signal import filtfilt, butter, find_peaks
from pwm import PWM
import sys
from time import sleep
import threading
import ipywidgets as widgets

In [2]:
minSpeed =  1000000 # 0 speed
zeroRot  =  1000000
maxRot   =   500000
rot = zeroRot


pwm0 = PWM(0)  # motor
# pwm0.export()
pwm0.period = 20000000
pwm0.duty_cycle = minSpeed

pwm1 = PWM(1)  # servo
# pwm1.export() 
pwm1.period = 20000000
pwm1.duty_cycle = zeroRot 

pwm0.enable = True
pwm1.enable = True

if not debug:
    print("waiting for moter")
    sleep(3)
    print("motor ready")

waiting for moter
motor ready


In [3]:
roi_perc = 0.8
peaks_threshold = 0.9
threshold_deviation = 0.15
speed = 1200000

# if debug:
#     speed = minSpeed

def set_r_s(r):
    global roi_perc
    roi_perc = r["new"]

def set_p_s(p):
    global peaks_threshold
    peaks_threshold = p["new"]
    
def set_t_s(t):
    global threshold_deviation
    threshold_deviation = t["new"]  
    
def set_s_s(s):
    print("called")
    global speed
    speed = s["new"] 
    

r_s = widgets.FloatSlider(value=0.8, min=0.0, max=1.0)
p_s = widgets.FloatSlider(value=0.9, min=0.0, max=1.0)
t_s = widgets.FloatSlider(value=0.15, min=0.0, max=1.0)
s_s = widgets.IntSlider(value=1200000, min=1000000, max=2000000, step=100000)

r_s.observe(set_r_s, "value")
p_s.observe(set_p_s, "value")
t_s.observe(set_t_s, "value")
s_s.observe(set_s_s, "value")
display(r_s, p_s, t_s, s_s)

FloatSlider(value=0.8, max=1.0)

FloatSlider(value=0.9, max=1.0)

FloatSlider(value=0.15, max=1.0)

IntSlider(value=1200000, max=2000000, min=1000000, step=100000)

In [4]:
camera = PiCamera()
print(speed)

1200000


In [5]:
def t_func():
    global rot
    
    global roi_perc
    global peaks_threshold
    global threshold_deviation
    global speed
    
    try:
        rawCapture = PiYUVArray(camera)
        stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)
        for frame in stream:
            b, a = butter(3, 0.007)
            I = frame.array[:, :, 0]
            rawCapture.truncate(0)

            mid_horizontal = int(I.shape[0]*roi_perc)
            mid_vertical = int(I.shape[1]/2)

            L = I[mid_horizontal, :]
            Lf = filtfilt(b, a, L)

            # normalize LF
            max_Lf = max(Lf)
            min_Lf = min(Lf)
            Lf = [val/(max_Lf - min_Lf) for val in Lf]

            p = find_peaks(Lf, peaks_threshold)

            closest_peak_at = -1
            distance_peak = 1000000
            for peak in p[0]:
                if abs(mid_vertical - peak) < distance_peak:
                    closest_peak_at = peak
                    distance_peak = abs(mid_vertical - peak)
            deviation = ((mid_vertical - closest_peak_at)*1.0)/mid_vertical

            if deviation < -1*threshold_deviation:
                print("go right")
                move = "right"
            elif -1*threshold_deviation <= deviation <= threshold_deviation:
                print("go straight")
                move = "straight"
            else:
                print("go left")
                move = "left"

            if debug:
                print("""
                    peaks: {0}
                    closest_peak_at: {1}
                    deviation: {2}
                    threshold_deviation: {3}
                    result:{4}
                """.format(str(p[0]), closest_peak_at, deviation, threshold_deviation, move))
                continue

            if move == 'left':
                rot = min(zeroRot + maxRot, rot+0.10*rot)
            elif move == 'right':
                rot = max(zeroRot -maxRot, rot-0.10*rot)
            else:
                rot = zeroRot

            print(speed, roi_perc, peaks_threshold, threshold_deviation)
            pwm0.duty_cycle = speed
            pwm1.duty_cycle = int(rot)
    except (KeyboardInterrupt, SystemExit):
        # bring the car on rest in any system failure
        print("Keyboard Interrup. Resetting Motor and Servo")
        pwm0.duty_cycle = minSpeed
        pwm1.duty_cycle = zeroRot
        stream.close()
        rawCapture.close()
        camera.close()
    except Exception as e:
        print(e)
        pwm0.duty_cycle = minSpeed
        pwm1.duty_cycle = zeroRot
        stream.close()
        rawCapture.close()
        camera.close()
    stream.close()
    rawCapture.close()
    camera.close()


In [6]:
t = threading.Thread(target = t_func)
t.start()

/home/pi/.virtualenvs/raceon/lib/python3.5/site-packages/picamera/encoders.py:544: PiCameraResolutionRounded: frame size rounded up from 720x480 to 736x480
  width, height, fwidth, fheight)))


In [ ]:
# t.kill()

In [7]:
sl = widgets.FloatSlider(value=5, min=0, max=10)
display(sl)

FloatSlider(value=5.0, max=10.0)

In [8]:
sl.value = 2.2

go straight
1200000 0.8 0.9 0.15


In [9]:
def f(s):
    print(s['new'])

sl.observe(f, names="value")

go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
go straight
1200000 0.8 0.9 0.15
